In [12]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [4]:
df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_balanced.csv")

In [5]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

In [39]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = to_categorical(train['sentiment'])

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [40]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size,input_length = maxlen))
model.add(Conv1D(filters = 50, kernel_size = 2, padding='same',activation = "relu"))
model.add(GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(2, activation="softmax"))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 130, 128)          768000    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 130, 50)           12850     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 780,952
Trainable params: 780,952
Non-trainable params: 0
_________________________________________________________________


In [41]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 165222 samples, validate on 41306 samples
Epoch 1/3
165222/165222 [==============================] - 79s 479us/step - loss: 0.1867 - acc: 0.9273 - val_loss: 0.1464 - val_acc: 0.9428
Epoch 2/3
165222/165222 [==============================] - 81s 490us/step - loss: 0.1123 - acc: 0.9580 - val_loss: 0.1410 - val_acc: 0.9466
Epoch 3/3
165222/165222 [==============================] - 87s 525us/step - loss: 0.0785 - acc: 0.9725 - val_loss: 0.1463 - val_acc: 0.9462


In [48]:
y_test = test["sentiment"]
list_sentences_test = test['Processed_Reviews']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)


51632
51632


In [54]:
print(np.argmax(prediction,axis = 1)[0])
print(y_test[0])

0
0


In [45]:
import numpy as np 
y_pred = np.argmax(prediction,axis = 1)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
#confusion_matrix(y_pred, y_test)

TypeError: Singleton array 452 cannot be considered a valid collection.